In [13]:
import numpy as np
from helpers import *

In [15]:
# You have to change the path for it to work

data_path = 'D:\\EPFL\\MA1\\Machine Learning\\Projet 1\\dataset'

#x_train_csv = 'D:\\EPFL\\MA1\\Machine Learning\\Projet 1\\dataset\\x_train.csv'
#y_train_csv = 'D:\\EPFL\\MA1\\Machine Learning\\Projet 1\\dataset\\y_train.csv'
#x_test_csv = 'D:\\EPFL\\MA1\\Machine Learning\\Projet 1\\dataset\\x_test.csv'

In [16]:
x_train, x_test, y_train, train_ids, test_ids = load_csv_data(data_path, sub_sample=False)

In [17]:
x_train[:5,:5]

array([[5.3000000e+01, 1.1000000e+01, 1.1162015e+07, 1.1000000e+01,
        1.6000000e+01],
       [3.3000000e+01, 1.2000000e+01, 1.2152015e+07, 1.2000000e+01,
        1.5000000e+01],
       [2.0000000e+01, 1.0000000e+01, 1.0202015e+07, 1.0000000e+01,
        2.0000000e+01],
       [4.2000000e+01, 6.0000000e+00, 6.1820150e+06, 6.0000000e+00,
        1.8000000e+01],
       [2.4000000e+01, 1.1000000e+01, 1.1062015e+07, 1.1000000e+01,
        6.0000000e+00]])